# Demo of Evaluation of Benchmarks

In [ ]:
from dbmsbenchmarker import *
import pandas as pd

import logging
logging.basicConfig(level=logging.INFO)

## Inspect Result Folder

In [ ]:
# path of folder containing experiment results
resultfolder = "/results/"

# create evaluation object for result folder
evaluate = inspector.inspector(resultfolder)

# list of all experiments in folder
evaluate.list_experiments
# dataframe of experiments
evaluate.get_experiments_preview()

## Pick an Experiment

In [ ]:
# last Experiment
code = evaluate.list_experiments[len(evaluate.list_experiments)-1]

# Specific Experiment
code = '1625203913'

# load it
evaluate.load_experiment(code)

## Load general properties into variables

In [ ]:
###################
##### general experiment properties
###################

# get experiment workflow
df = evaluate.get_experiment_workflow()

# get workload properties
workload_properties = evaluate.get_experiment_workload_properties()
print(workload_properties['name'])

In [ ]:
evaluator.pretty(evaluate.get_experiment_connection_properties('Citus-1-1'))

In [ ]:
query_properties = evaluate.get_experiment_query_properties()
print(query_properties[str(numQuery)]['config']['title'])
print(query_properties[str(numQuery)]['config']['query'])

In [ ]:
# list queries
list_queries = evaluate.get_experiment_list_queries()

# list connections
list_nodes = evaluate.get_experiment_list_nodes()
list_dbms = evaluate.get_experiment_list_dbms()
list_connections = evaluate.get_experiment_list_connections()
list_connections_node = evaluate.get_experiment_list_connections_by_node()
list_connections_dbms = evaluate.get_experiment_list_connections_by_dbms()
list_connections_clients = evaluate.get_experiment_list_connections_by_connectionmanagement('numProcesses')
list_connections_gpu = evaluate.get_experiment_list_connections_by_hostsystem('GPU')

# fix some examples:
# first connection, first query, first run
connection = list_connections[0]
numQuery = 1
numRun = 0

pd.set_option("max_rows", None)

## Show some Measures

### Hardware Metrics

In [ ]:
# list all available metrics
pd.DataFrame(monitor.metrics.metrics).T

In [ ]:
# get hardware metrics for loading test
df = evaluate.get_loading_metrics('total_cpu_memory')
df = evaluate.get_loading_metrics('total_cpu_util_s')
df = df.T.max().sort_index() - df.T.min().sort_index() # compute difference of counter
print("CPU of Ingestion (via counter)")
print(df)

In [ ]:
df = evaluate.get_loading_metrics('total_cpu_util')
df = df.T.sum() # computer sum of rates
print("CPU of Ingestion (via rate)")
print(df)

In [ ]:
# get hardware metrics per stream
df = evaluate.get_streaming_metrics('total_cpu_memory')
df = evaluate.get_streaming_metrics('total_cpu_util_s')
df = df.T.max().sort_index() - df.T.min().sort_index() # compute difference of counter
print("CPU of Stream (via counter)")
print(df)

In [ ]:
df = evaluate.get_streaming_metrics('total_cpu_util')
df = df.T.sum() # computer sum of rates
print("CPU of Stream (via rate)")
print(df)

### Measures, show as table per query

In [ ]:
evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='throughput', name='run', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='latency', name='run', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='timer', name='connection', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='timer', name='execution', query_aggregate='Mean').T

In [ ]:
evaluate.get_aggregated_query_statistics(type='timer', name='datatransfer', query_aggregate='Mean').T

In [ ]:
# measures, show as heatmap (normalized)
evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='factor').T

In [ ]:
# example: filter DBMS
list_filter_dbms = list_connections_dbms['Citus']
evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='factor', dbms_filter=list_filter_dbms).T

In [ ]:
# size of result sets
evaluate.get_total_resultsize()

In [ ]:
# relative size of result sets (minimum=100%), show as heatmap
evaluate.get_total_resultsize_normalized()

In [ ]:
# if there was an error, show as heatmap
evaluate.get_total_errors()

In [ ]:
# if there was a warning, show as heatmap
evaluate.get_total_warnings()

In [ ]:
# total times benchmarking a query took, show as table
evaluate.get_total_times().T

In [ ]:
# same, normalized to 100% per query, show as stacked area plot
evaluate.get_total_times_normalized().T

In [ ]:
# same, normed to relative to best per query, show as heatmap
evaluate.get_total_times_relative().T

# some dataframes

* showing metrics per dbms by aggregation
* from measures, show as bar plot

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='timer', name='run', query_aggregate='Mean', total_aggregate='Mean')

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='throughput', name='run', query_aggregate='Mean', total_aggregate='Geo')

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='timer', name='run', query_aggregate='factor', total_aggregate='Geo')

In [ ]:
evaluate.get_aggregated_experiment_statistics(type='monitoring', name='total_cpu_memory', query_aggregate='Mean', total_aggregate='Mean')

In [ ]:
# average data obtained from monitoring
# average per second - this differs from mean of mean, because not all queries have the same duration
evaluate.get_survey_monitoring().T

In [ ]:
# host data obtained from config
evaluate.get_survey_hostdata()
# example for merging two dataframes (by index)
df1 = evaluate.get_survey_monitoring()
df2 = evaluate.get_survey_hostdata()
tools.dataframehelper.merge(df1, df2)

In [ ]:
evaluate.get_survey_hostdata()

## Per query details

In [ ]:
# get measures of a query
# 
# df1 = measures for plot, boxplot, histogramm
# df2 = statistics as table
df1, df2 = evaluate.get_measures_and_statistics(numQuery, type='timer', name='execution')
print(df1)
df2

In [ ]:
evaluate.get_measures_and_statistics(numQuery, type='timer', name='execution')
evaluate.get_measures_and_statistics(numQuery, type='timer', name='datatransfer')
evaluate.get_measures_and_statistics(numQuery, type='timer', name='run')
evaluate.get_measures_and_statistics(numQuery, type='timer', name='session')
evaluate.get_measures_and_statistics(numQuery, type='throughput', name='session')

In [ ]:

# colors by dbms
list_connections_dbms = evaluate.get_experiment_list_connections_by_dbms()
connection_colors = evaluate.get_experiment_list_connection_colors(list_connections_dbms)
#connection_colors = evaluate.get_experiment_list_connection_colors(list_connections_node)

numQuery=1
df1,df2=evaluate.get_measures_and_statistics(numQuery, type='timer', name='run', warmup=0)
# using colors in matplotlib plots
#import matplotlib.pyplot as plt
#df1.T.plot(color=[connection_colors.get(x, '#333333') for x in df1.T.columns])
#plt.show()


# Some plotly figures
import plotly.graph_objects as go

# Plots
fig = go.Figure()
for i in range(len(df1.index)):
    t = fig.add_trace(go.Scatter(x=df1.T.index, y=df1.iloc[i], name=df1.index[i], line=dict(color=connection_colors[df1.index[i]], width=1)))

fig.show()


In [ ]:
# Heatmap
df = evaluate.get_aggregated_query_statistics(type='timer', name='run', query_aggregate='factor')
df = df.sort_index()
df_2=df.applymap(lambda x: ("%.2f" % x))
fig = go.Figure(data=[go.Heatmap(z=df_2.T.values.tolist(),x=df_2.index, y=df_2.columns,colorscale='Reds')])
# optionally fixed size
#t = fig.update_layout(autosize=False,height=2000)
fig.show()

In [ ]:
# Bar
df1, df2 = evaluate.get_measures_and_statistics(numQuery, type='timer', name='run')
df = tools.dataframehelper.collect(df2, 'Mean', 'timer_run_mean')
fig = go.Figure()
for i in range(len(df.index)):
    t = fig.add_trace(go.Bar(x=[df.index[i]], y=df.iloc[i], name=df.index[i], marker=dict(color=connection_colors[df.index[i]])))

fig.show()

In [ ]:
# Boxplots
fig = go.Figure()
for i in range(len(df1.index)):
    t = fig.add_trace(go.Box(y=df1.iloc[i], name=df1.index[i], line=dict(color=connection_colors[df1.index[i]], width=1), boxmean='sd'))

fig.show()
